In [33]:
# === Setup and Libraries ===
import requests
from datetime import datetime, timedelta
import pandas as pd
from pytz import utc
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator
import ipywidgets as widgets
from IPython.display import display, clear_output

# === Helper: Convert string date to UTC datetime ===
def convert_date_to_utc_datetime(date_string):
    return datetime.strptime(date_string, "%d-%b-%Y").replace(tzinfo=utc)

# === Helper: Split large date range into smaller chunks (AMFI limit) ===
def split_date_range(start_date_str, end_date_str, max_duration=90):
    start_date = datetime.strptime(start_date_str, "%d-%b-%Y")
    end_date = datetime.strptime(end_date_str, "%d-%b-%Y")
    ranges = []
    current = start_date
    while current <= end_date:
        chunk_end = min(current + timedelta(days=max_duration - 1), end_date)
        ranges.append((current, chunk_end))
        current = chunk_end + timedelta(days=1)
    return ranges

# === Fetch AMFI NAV data function ===
def fetch_amfi_data(start_date_str, end_date_str):
    nav_list = []
    for start, end in split_date_range(start_date_str, end_date_str):
        url = f"https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?&frmdt={start.strftime('%d-%b-%Y')}&todt={end.strftime('%d-%b-%Y')}"
        response = requests.get(url)
        print(f"Fetched: {start.strftime('%d-%b-%Y')} to {end.strftime('%d-%b-%Y')}")
        lines = response.text.split('\r\n')
        Structure = Category = Sub_Category = amc = ""
        j = 1
        for line in lines[1:]:
            split = line.split(";")
            if j == len(lines) - 1:
                break
            if split[0] == "":
                if lines[j] == lines[j + 1]:
                    sch_cat = lines[j - 1].split("(")
                    sch_cat[-1] = sch_cat[-1][:-2].strip()
                    sch_cat = [s.strip() for s in sch_cat]
                    if "-" in sch_cat[1]:
                        sub = sch_cat[1].split("-")
                        sch_cat.pop(-1)
                        sch_cat += [s.strip() for s in sub]
                    else:
                        sch_cat += ["", sch_cat.pop(-1)]
                    Structure, Category, Sub_Category = sch_cat[:3]
                elif "Mutual Fund" in lines[j + 1]:
                    amc = lines[j + 1]
            elif len(split) > 1:
                try:
                    code = int(split[0].strip())
                    name = split[1].strip()
                    dg = "Growth" if "growth" in name.lower() else "IDCW" if "idcw" in name.lower() or "dividend" in name.lower() else ""
                    inv_src = "Direct" if "direct" in name.lower() else "Regular" if "regular" in name.lower() else ""
                    nav = float(split[4].strip()) if split[4].strip() else None
                    date = convert_date_to_utc_datetime(split[7].strip())
                    nav_list.append({
                        "Structure": Structure,
                        "Category": Category,
                        "Sub_Category": Sub_Category,
                        "AMC": amc,
                        "Code": code,
                        "Name": name,
                        "Source": inv_src,
                        "Option": dg,
                        "Date": date,
                        "NAV": nav
                    })
                except:
                    pass
            j += 1
    return pd.DataFrame(nav_list)

# === Global NAV DataFrame ===
df_nav = pd.DataFrame()

# === UI Elements ===
fetch_from_date = widgets.DatePicker(description='Fetch From:', value=datetime(2025, 4, 1))
fetch_to_date = widgets.DatePicker(description='Fetch To:', value=datetime(2025, 6, 30))
fetch_button = widgets.Button(description="📥 Fetch Data", button_style='info')

amc_dropdown = widgets.Dropdown(options=[], description='AMC:')
scheme_dropdown = widgets.Dropdown(description='Scheme:')
from_date = widgets.DatePicker(description='From:')
to_date = widgets.DatePicker(description='To:')
sma_input_1 = widgets.IntText(value=50, description='SMA 1:')
sma_input_2 = widgets.IntText(value=100, description='SMA 2:')
sma_input_3 = widgets.IntText(value=200, description='SMA 3:')
plot_button = widgets.Button(description="📊 Plot Chart", button_style='success')
output = widgets.Output()

# === Fetch Button Logic ===
def on_fetch_clicked(b):
    with output:
        clear_output()
        if not fetch_from_date.value or not fetch_to_date.value:
            print("⚠️ Select both fetch start and end dates.")
            return

        start_str = fetch_from_date.value.strftime('%d-%b-%Y')
        end_str = fetch_to_date.value.strftime('%d-%b-%Y')

        print(f"📦 Fetching AMFI data from {start_str} to {end_str}...")
        global df_nav
        df_nav = fetch_amfi_data(start_str, end_str)
        if df_nav.empty:
            print("❌ No data fetched.")
            return

        df_nav['Date'] = pd.to_datetime(df_nav['Date']).dt.tz_localize(None)
        amc_dropdown.options = sorted(df_nav['AMC'].dropna().unique())
        print(f"✅ Data fetched and loaded with {len(df_nav)} records.")

fetch_button.on_click(on_fetch_clicked)

# === Dropdown Interactions ===
def on_amc_change(change):
    selected_amc = change['new']
    schemes = sorted(df_nav[df_nav['AMC'] == selected_amc]['Name'].unique())
    scheme_dropdown.options = schemes
amc_dropdown.observe(on_amc_change, names='value')

def on_scheme_change(change):
    df = df_nav[df_nav['Name'] == change['new']]
    if not df.empty:
        from_date.value = df['Date'].min().to_pydatetime()
        to_date.value = df['Date'].max().to_pydatetime()
scheme_dropdown.observe(on_scheme_change, names='value')

# === Plotting Logic ===
def plot_graph(b):
    with output:
        clear_output()
        if not scheme_dropdown.value or not from_date.value or not to_date.value:
            print("⚠️ Select AMC, Scheme, and Dates")
            return

        df = df_nav[df_nav['Name'] == scheme_dropdown.value].copy()
        df = df[(df['Date'] >= pd.to_datetime(from_date.value)) & (df['Date'] <= pd.to_datetime(to_date.value))]

        if df.empty:
            print("❌ No data for selected range")
            return

        # Indicators
        df['RSI_14'] = RSIIndicator(close=df['NAV'], window=14).rsi()
        macd = MACD(close=df['NAV'], window_slow=26, window_fast=12, window_sign=9)
        df['MACD'], df['MACD_Signal'] = macd.macd(), macd.macd_signal()

        sma_cols = []
        for sma_input in [sma_input_1, sma_input_2, sma_input_3]:
            if sma_input.value > 1:
                col = f"SMA_{sma_input.value}"
                df[col] = SMAIndicator(close=df['NAV'], window=sma_input.value).sma_indicator()
                sma_cols.append(col)

        # Plot
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                            subplot_titles=["NAV + SMAs", "RSI (14)", "MACD"])
        fig.add_trace(go.Scatter(x=df['Date'], y=df['NAV'], name="NAV", line=dict(color="cyan")), row=1, col=1)
        for col in sma_cols:
            fig.add_trace(go.Scatter(x=df['Date'], y=df[col], name=col, line=dict(dash='dot')), row=1, col=1)
        fig.add_trace(go.Scatter(x=df['Date'], y=df['RSI_14'], name="RSI", line=dict(color="violet")), row=2, col=1)
        fig.add_shape(type='line', x0=df['Date'].min(), x1=df['Date'].max(), y0=70, y1=70, line=dict(dash="dot", color="red"), row=2, col=1)
        fig.add_shape(type='line', x0=df['Date'].min(), x1=df['Date'].max(), y0=30, y1=30, line=dict(dash="dot", color="green"), row=2, col=1)
        fig.add_trace(go.Scatter(x=df['Date'], y=df['MACD'], name="MACD", line=dict(color="aqua")), row=3, col=1)
        fig.add_trace(go.Scatter(x=df['Date'], y=df['MACD_Signal'], name="Signal", line=dict(dash='dot', color="white")), row=3, col=1)
        fig.update_layout(title=f"{scheme_dropdown.value} NAV Chart", template="plotly_dark", height=900)
        fig.show()

plot_button.on_click(plot_graph)

# === Layout ===
display(widgets.VBox([
    widgets.HBox([fetch_from_date, fetch_to_date, fetch_button]),
    amc_dropdown,
    scheme_dropdown,
    widgets.HBox([from_date, to_date]),
    widgets.HBox([sma_input_1, sma_input_2, sma_input_3]),
    plot_button,
    output
]))
